In [1]:
import pandas as pd
import numpy as np
import re
import time
import schedule
import datetime
import gc

In [2]:
# neg-1, neu-2, pos-3  
def senti_number(txt):
    if txt == 'Negative':
        txt = re.sub(r'Negative', 1, txt)
    elif txt == 'Neutral':
        txt = re.sub(r'Neutral', 2, txt)
    else:
        txt = re.sub(r'Positive', 3, txt)
    
    return txt

In [3]:
# 感情の数値化
# 引数：slist
def assign_section(df,timespan):
    df['Sentiment']  = df['Sentiment'].apply(senti_number)
    
    return df

In [4]:
# UTC -> UNIX
def utc_to_unix(txt):
    utc = datetime.datetime.strptime(txt, '%Y-%m-%dT%H:%M:%S.000Z')
    
    return utc.timestamp()  

In [5]:
# 区間ナンバー計算(2021-01-01T00:00を区間0とする)
def culc_section(unix,timespan):
    section = int((unix - 1609426800)/timespan)
    
    return section

In [6]:
# 区間ナンバー割り当て
# 引数：slist,BTCUSDT
def assign_section(df,timespan):
    tlis = {'1d':24*60*60, '12h':12*60*60, '4h':4*60*60, '1h':60*60, '30m':30*60, '15m':15*60, '5m':5*60}

    if timespan in tlis:
        s = tlis[timespan]
    else:
        print("KeyError")
        
    df['created_at'] = df['created_at'].apply(utc_to_unix)
    df['created_at'] = df['created_at'].apply(culc_section,args=(s,))
    
    return df

In [7]:
# 区間ごとにcomの平均をカウント
# 引数：slist,elist
def com_counter(dfs,dfe):
    df_groupby = dfs.groupby("created_at") 
    dfg = df_groupby.mean()
    dfe['com_ave'] = dfg['Compound']
    return dfe

In [8]:
# pos,neg,newを区間ごとにカウント
# 引数：slist,elist
def senti_counter(dfs,dfe):    
    tmp_df = dfs.value_counts(sort=False).reset_index(name='count')
    
    n = tmp_df.shape[0]
    k = int(n/3)

    df3 = [tmp_df.loc[i:i+k-1, :] for i in range(0, n, k)]
    
    dfe['neg_count'] = df3[0]['count']
    dfe['neu_count'] = df3[1]['count']
    dfe['pos_count'] = df3[2]['count']
    
    return dfe

In [9]:
#Tweet数をカウント
def tweet_counter(df):
    df['tweet_count']=df['pos_count']+df['neu_count']+df['tweet_count']
    
    return df

In [ ]:
# ラベル付け(トレンド=始値-終値)

In [ ]:
# main
month = int(input("monthを入力:"))
timespan = str(input("timespanを入力:"))

df  = pd.DataFrame(columns = ['section','neg_count','neu_count','pos_count','com_ave','tweet_count','start_price','trend'])
df1 = pd.read_csv(f'tweet-sentiment/2021-0{month}_slist.csv')
df2 = pd.read_csv(f'binance/BTCUSDT/{timespan}/BTCUSDT-{timespan}-2021-0{month}.csv', usecols = ['unix','open'])






# メモリ開放
del df,df1,df2
gc.collect()

In [10]:
# main test
df = pd.read_csv('tweet-sentiment/2021-01_slist.csv', usecols = ['Compound','created_at'])
df = assign_section(df,'1d')

df1  = pd.DataFrame(columns = ['section','neg_count','neu_count','pos_count','com_ave','tweet_count','start_price','trend'])
df1 = com_counter(df,df1)

df = pd.read_csv('tweet-sentiment/2021-01_slist.csv', usecols = ['Sentiment','created_at'])
df = assign_section(df,'1d')

df1 = senti_counter(df,df1)
print(df1)


           section  neg_count  neu_count  pos_count   com_ave tweet_count  \
created_at                                                                  
0              NaN       4549        NaN        NaN  0.178786         NaN   
1              NaN      11070        NaN        NaN  0.129637         NaN   
2              NaN      10091        NaN        NaN  0.157232         NaN   
3              NaN       8207        NaN        NaN  0.173041         NaN   
4              NaN       5985        NaN        NaN  0.222922         NaN   
5              NaN       7721        NaN        NaN  0.177938         NaN   
6              NaN      10902        NaN        NaN  0.173976         NaN   
7              NaN       9291        NaN        NaN  0.213337         NaN   
8              NaN       6723        NaN        NaN  0.228476         NaN   
9              NaN       7297        NaN        NaN  0.172320         NaN   
10             NaN      17923        NaN        NaN  0.095024         NaN   

In [33]:
tmp_df = df.value_counts(sort=False).reset_index(name='count')

In [34]:
print(tmp_df)

   Sentiment  created_at  count
0   Negative           0   4557
1   Negative           1  11078
2   Negative           2  10097
3   Negative           3   8218
4   Negative           4   5995
..       ...         ...    ...
88  Positive          26  19290
89  Positive          27  23560
90  Positive          28  33495
91  Positive          29  19680
92  Positive          30  18328

[93 rows x 3 columns]


In [ ]:


#dfe['section'] = dfg['created_at']

In [37]:
    print(dfg)
    dfg.to_csv('csv-test/test_dfg.csv')

,Sentiment,created_at,count
0,Negative,0,4557
1,Negative,1,11078
2,Negative,2,10097
3,Negative,3,8218
4,Negative,4,5995
5,Negative,5,7739
6,Negative,6,10910
7,Negative,7,9299
8,Negative,8,6730
9,Negative,9,7311


,Sentiment,created_at,count
31,Neutral,0,12935
32,Neutral,1,33387
33,Neutral,2,25525
34,Neutral,3,22073
35,Neutral,4,19766
36,Neutral,5,25110
37,Neutral,6,34246
38,Neutral,7,27042
39,Neutral,8,17838
40,Neutral,9,18688


,Sentiment,created_at,count
62,Positive,0,13960
63,Positive,1,28038
64,Positive,2,26968
65,Positive,3,23711
66,Positive,4,23587
67,Positive,5,25068
68,Positive,6,34022
69,Positive,7,33192
70,Positive,8,25581
71,Positive,9,20673


In [68]:
# メモリ開放
del df1,df
gc.collect()

307

In [88]:
'''
def div(unix):
    unix = int(unix/1000)
    return unix

df = pd.read_csv('binance/BTCUSDT/15m/BTCUSDT-15m-2021-01.csv', header=None)
df.columns = ['unix',
                'open',
                'high',
                'low',
                'close',
                'volume',
                'close_time',
                'quote_asset_volume',
                'number_of_trades',
                'taker_buy_base_asset_volume',
                'taker_buy_quote_asset_volume',
                'ignore']
df['unix'] = df['unix'].apply(div)
df.drop(columns=['close_time'], inplace=True)
df.to_csv('binance/BTCUSDT/15m/BTCUSDT-15m-2021-01.csv', index=False)
del df
gc.collect()

'''

17